In [29]:
import numpy as np
import pandas as pd
from pymatch import Matcher
import sklearn.linear_model

In [172]:
def multivariate_df(n_samples, mean, cov):
    
    data = np.random.multivariate_normal(mean, cov, n_samples)
    
    cols = col_name_gen(len(mean), 'c')
    df = pd.DataFrame(data, columns=cols)
    info = {
        'distribution': 'Gaussian',
        'mean' : mean,
        'covariance' : cov,
        'num_samples' : n_samples
    }
    return df, info

def col_name_gen(num_cols, common_name):
    common_name_list = [common_name]*num_cols
    num_string_list = [num_string for num_string in map(
        lambda num: str(num), [num for num in range(num_cols)]
    )]
    res_list = [a + b for a, b in zip(common_name_list, num_string_list)]
    return res_list


def pMSE(dataframe, ind_var):
    #This should be implemented with multiple models chosen by some variable
    #For now it will be done with logistic regression as there is an analytical solutionn to the null value.
    #ind_var is the name of the indicator variable
    model = sklearn.linear_model.LogisticRegression(penalty = 'none').fit(dataframe.drop(ind_var, axis = 1), dataframe[ind_var])
    prediction = model.predict_proba(dataframe.drop(ind_var, axis = 1))
    c = sum(dataframe[ind_var]==0)/len(dataframe)
    pmse = (sum((prediction[:,1] - 0.5)**2))/len(dataframe)
    return pmse
    
def null_pmse_est(dataframe, ind_var, n_iter):
    
    df = dataframe.copy()
    pmse = 0
    
    for i in range(n_iter):
        if (i %5 == 0):
            print('estimation run no: ' + str(i)) 
        df[ind_var] = df[ind_var].sample(frac = 1).reset_index(drop = True)
        pmse += pMSE(df, ind_var)
        if (i == 1000):
            break 
    pmse /= n_iter
    return pmse

def df_concat_ind(real_df, gen_df, ind = 'ind'):
    
    r_df = real_df.copy()
    g_df = gen_df.copy()
    df = pd.concat((r_df, gen_df), axis=0)
    
    ind_col = pd.DataFrame(np.ones( (len(df), 1) ) , columns= [ind])
    df = pd.concat((df, ind_col), axis = 1)
    
    return df
    
        


In [173]:

testing_df, info_1 = multivariate_df(100, [0,0], np.eye(2))
df2, info_2 = multivariate_df(100, [1,1], [[1,0.5],[0.5,1]])
testing_df = pd.concat([testing_df, pd.DataFrame(np.ones(len(testing_df)), columns = ['ind'])], axis = 1) 
df2 = pd.concat([df2, pd.DataFrame(np.zeros(len(df2)), columns = ['ind'])], axis = 1) 
df = pd.concat([testing_df, df2])
df = df.reset_index(drop=True)

print(null_pmse_est(df, 'ind', 1000000))
print(  pMSE(df, 'ind'))

estimation run no: 0
estimation run no: 5
estimation run no: 10
estimation run no: 15
estimation run no: 20
estimation run no: 25
estimation run no: 30
estimation run no: 35
estimation run no: 40
estimation run no: 45
estimation run no: 50
estimation run no: 55
estimation run no: 60
estimation run no: 65
estimation run no: 70
estimation run no: 75
estimation run no: 80
estimation run no: 85
estimation run no: 90
estimation run no: 95
estimation run no: 100
estimation run no: 105
estimation run no: 110
estimation run no: 115
estimation run no: 120
estimation run no: 125
estimation run no: 130
estimation run no: 135
estimation run no: 140
estimation run no: 145
estimation run no: 150
estimation run no: 155
estimation run no: 160
estimation run no: 165
estimation run no: 170
estimation run no: 175
estimation run no: 180
estimation run no: 185
estimation run no: 190
estimation run no: 195
estimation run no: 200
estimation run no: 205
estimation run no: 210
estimation run no: 215
estimation

KeyboardInterrupt: 

In [160]:
df.loc[:,'ind'] = df.loc[:,'ind'].sample(frac=1).reset_index(drop= True)
df


,c0,c1,ind
0,0.658014,0.565689,1.0
1,0.139290,0.802943,0.0
2,2.192360,-0.230403,1.0
3,-0.212145,0.100580,1.0
4,-0.190139,-0.972706,0.0
5,0.824050,-0.254794,1.0
6,-0.145733,0.920950,0.0
7,-2.314750,-1.147806,0.0
8,-0.319632,0.168714,1.0
9,0.039004,-0.790089,1.0


In [19]:



# testing_df, info_1 = multivariate_df(100, [0,0], np.eye(2))
# df2, info_2 = multivariate_df(100, [0,0], [[1,0.5],[0.5,1]])
# testing_df = pd.concat([testing_df, pd.DataFrame(np.ones(len(testing_df)), columns = ['ind'])], axis = 1) 
# df2 = pd.concat([df2, pd.DataFrame(np.zeros(len(df2)), columns = ['ind'])], axis = 1) 
# df_control = pd.concat([testing_df, df2])
# df_control = df.sample(frac = 1)
# df_control = df.reset_index(drop = True)

test = df[df['ind'] == 1]
control = df[df['ind'] == 0]


In [20]:

m = Matcher.Matcher(test, control, 'ind')

Formula:
ind ~ c0+c1
n majority: 100
n minority: 100


In [18]:
m.fit_scores(balance=True, nmodels = 100)

Fitting Models on Balanced Samples: 1\100Error: Unable to coerce to Series, length must be 1: given 200
Fitting Models on Balanced Samples: 1\100Error: Unable to coerce to Series, length must be 1: given 200
Fitting Models on Balanced Samples: 1\100Error: Unable to coerce to Series, length must be 1: given 200
Fitting Models on Balanced Samples: 1\100Error: Unable to coerce to Series, length must be 1: given 200
Fitting Models on Balanced Samples: 1\100Error: Unable to coerce to Series, length must be 1: given 200

Average Accuracy: nan%
